In [1]:
import random
import math

from IPython.display import IFrame
IFrame("Blatt06.pdf", width=1000, height=500)

In [2]:
def is_square(i: int) -> bool:
    return i == math.isqrt(i) ** 2


def fermat_factorization(n):
    print(f"Fermat factorization of {n}")
    x = math.ceil(math.sqrt(n)) - 1
    print(f"{x ** 2} = {x}^2")
    d = 2
    i = 0
    while not is_square(d):
        x += 1
        d = x ** 2 - n
        print(f"d{i} = {x}^2 - {n} = {d}")
        i += 1
    d = int(math.sqrt(d) + 0.5)
    print(f"{d ** 2} = {d}^2")
    return x+d, x-d


print(fermat_factorization(6161))
print(fermat_factorization(2923))

Fermat factorization of 6161
6084 = 78^2
d0 = 79^2 - 6161 = 80
d1 = 80^2 - 6161 = 239
d2 = 81^2 - 6161 = 400
400 = 20^2
(101, 61)
Fermat factorization of 2923
2916 = 54^2
d0 = 55^2 - 2923 = 102
d1 = 56^2 - 2923 = 213
d2 = 57^2 - 2923 = 326
d3 = 58^2 - 2923 = 441
441 = 21^2
(79, 37)


# 6.1 b)

x muss die Werte von $\sqrt{pq}$ bis $\frac{p+q}{2}$ durchlaufen.\
Bei 499- bzw. 501-Bit Zahlen $p, q$ liegt die Differenz circa bei $\frac{(0.5*2^{500}\ +\ 2*2^{500})}{2}\ -\ 2^{500}\ =\ \frac{1}{4}* 2^{500}\ =\ 2^{498}$

In [7]:
def mod_exp(a, x, n):
    d = 1
    for i in bin(x)[2:]:
        d = (d * d) % n
        if int(i) == 1: d = (d * a) % n
    return d


def ggT(a,b):
    if b == 0: return a
    return ggT(b, a % b)


def p_1_pollard(n, a=2):
    B = 1
    g = 1
    while g == 1:
        B += 1
        a = mod_exp(a,B,n)
        g = ggT(a-1, n)
        print(f"B: {B}, a: {a}, g: {g}")
    if g == n:
        return "failure"
    return g


def gen_prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return list(set([(x, factors.count(x)) for x in factors]))


print(gen_prime_factors(103-1))
print(gen_prime_factors(241-1))

p_1_pollard(103*241)

[(3, 1), (17, 1), (2, 1)]
[(3, 1), (2, 4), (5, 1)]
B: 2, a: 4, g: 1
B: 3, a: 64, g: 1
B: 4, a: 21691, g: 241


241

In [14]:
def pollard_rho(n, z0=20):
    i = 0
    x = z0
    y = z0
    g = 1
    while g == 1:
        x = (x ** 2 + 1) % n
        y = (y ** 2 + 1) % n
        print(y)
        y = (y ** 2 + 1) % n
        g = ggT(x - y, n)
        i += 1
        print(f"i: {i}, z{i}: {x}, z{2*i}: {y}, g: {g}")
    if g == n: return "failure"
    return g

print(pollard_rho(1219))

401
i: 1, z1: 401, z2: 1113, g: 1
266
i: 2, z2: 1113, z4: 55, g: 23
23


In [17]:
def extggT(a,b):
    if b == 0: return a,1,0
    d,x,y = extggT(b, a % b)
    return d,y,x-(a//b)*y


def babystep_giantstep(n, a=3, y=11):
    lower_sqrt_n = math.floor(math.sqrt(n))
    higher_sqrt_n = math.ceil(math.sqrt(n))
    v = (a ** higher_sqrt_n) % n
    L = dict([((v ** x) % n, x) for x in range(lower_sqrt_n + 1)])
    u = extggT(a, n)[1] % n
    print(lower_sqrt_n, higher_sqrt_n, v, u)
    print(L)
    for x2 in range(lower_sqrt_n):
        z = (y * (u ** x2)) % n
        print(f"z: {z}")
        if z in L:
            print(f"x1*higher_sqrt(n)+x2 = {L[z]}*{higher_sqrt_n}+{x2}")
            return L[z] * higher_sqrt_n + x2
        

def discrete_log(base, modulo):
    return [(x, (base ** x) % modulo) for x in range (modulo)]
        

print(discrete_log(3, 137)[122])
babystep_giantstep(137)

(122, 11)
11 12 18 46
{1: 0, 18: 1, 50: 2, 78: 3, 34: 4, 64: 5, 56: 6, 49: 7, 60: 8, 121: 9, 123: 10, 22: 11}
z: 11
z: 95
z: 123
x1*higher_sqrt(n)+x2 = 10*12+2


122

In [6]:
def phi(n):
    prime_factors = gen_prime_factors(n)
    phi = n
    for p, e in prime_factors:
        phi = phi * (1 - 1/p)
    return int(phi)


def RSA_find_smallest_e(n):
    phi_n = phi(n)
    for e in range(2, phi_n):
        if ggT(e, phi_n) == 1:
            return e
    

def RSA_d_to_e(e, n):
    phi_n = phi(n)
    return extggT(e, phi_n)[1] % phi_n


def RSA_sign_m(m, d, n):
    return m, mod_exp(m, d, n)


def RSA_blend_m(m, e, z, n):
    m_new = m*mod_exp(z,e,n) % n
    return m_new


n = 11 * 13
e = RSA_find_smallest_e(n=n)
d = RSA_d_to_e(e=e, n=n)
z = 5
m=9
print(f"n: {n}")
print(f"e: {e}")
print(f"d: {d}")
print(f"m: {m}")
print(f"z: {z}")
_, u = RSA_sign_m(m=m, d=d, n=n)
m_new = RSA_blend_m(m=m, e=e, z=z, n=n)
_, u_new = RSA_sign_m(m=m_new, d=d, n=n)
print(f"u: {m}^{d} mod {n} = {u}")
print(f"Signature: ({m}, {u})")
print(f"m_new: {m}*{z}^{e} mod {n} = {m_new}")
print(f"u_new: {m_new}^{d} mod {n} = {u_new}")
print(f"Signature: ({m_new}, {u_new})")
z_inv = extggT(z, n)[1] % n
u_reverse = (u_new * z_inv) % n
print(f"z_inv: {z_inv}")
print(f"u_reverse: {u_reverse}")
print(f"u == u_reverse: {u == u_reverse}")

n: 143
e: 7
d: 103
m: 9
z: 5
u: 9^103 mod 143 = 113
Signature: (9, 113)
m_new: 9*5^7 mod 143 = 137
u_new: 137^103 mod 143 = 136
Signature: (137, 136)
z_inv: 86
u_reverse: 113
u == u_reverse: True
